In [4]:
import pandas as pd

In [5]:
students_per_course = 30

class Student():
    def __init__(self, student_id, preferences):
        self.id = student_id
        self.preferences = preferences
    
    attending = []

class Course():
    def __init__(self, course_id):
        self.id = course_id
        
    max_students = students_per_course

In [6]:
Julen = Student('A20512110', [0, 1, 5])

print(Julen.id)
print(Julen.preferences)
print(Julen.attending)

Machine_Learning = Course('CS538')

print(Machine_Learning.id)
print(Machine_Learning.max_students)

A20512110
[0, 1, 5]
[]
CS538
30
